네이버영화평점
==
- 감성분석
- 네이버 영화평점 데이터(https://github.com/e9t/nsmc)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/swkim01/DSAC3/blob/main/gg-53-네이버영화평점.ipynb"><img src="https://github.com/swkim01/DSAC3/raw/main/colab_logo_32px.png" />구글 코랩에서 실행</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/swkim01/DSAC3/blob/main/gg-53-네이버영화평점.ipynb"><img src="https://github.com/swkim01/DSAC3/raw/main/GitHub-Mark-32px.png" />깃헙에서 소스 보기</a>
  </td>
</table>

# 로지스틱회귀를 이용한 감성분석

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 11.2MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 460kB 41.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [2]:
# 네이버 영화 평점 데이터 다운로드
#!curl -L https://bit.ly/2X9Owwr -o ratings_train.txt
!curl -L https://github.com/swkim01/DSAC3/raw/main/ratings_train.txt -o ratings_train.txt
#!curl -L https://bit.ly/2WuLd5I -o ratings_test.txt
!curl -L https://github.com/swkim01/DSAC3/raw/main/ratings_test.txt -o ratings_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   136  100   136    0     0    327      0 --:--:-- --:--:-- --:--:--   327
100 14.0M  100 14.0M    0     0  7191k      0  0:00:02  0:00:02 --:--:-- 38.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   135  100   135    0     0    125      0  0:00:01  0:00:01 --:--:--   125
100 4827k  100 4827k    0     0  2335k      0  0:00:02  0:00:02 --:--:-- 29.0M


In [3]:
import konlpy
import pandas as pd
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import make_pipeline
# import pickle
# import os.path

# 데이터 로드
df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)

df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [12]:
text_train = df_train['document'].values
y_train = df_train['label'].values
text_test = df_test['document'].values
y_test = df_test['label'].values

In [13]:
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

In [14]:
twitter_tag = Twitter()

cv = TfidfVectorizer(tokenizer=twitter_tokenizer, min_df=5)

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [15]:
lr = LogisticRegression()

x_train = cv.fit_transform(text_train)
x_test = cv.transform(text_test)
result = lr.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [16]:
len(cv.vocabulary_)

22414

In [17]:
feature_names = cv.get_feature_names()
print(feature_names[3000:3010])

['기억나는게', '기억나는데', '기억나서', '기억나요', '기억나지', '기억난다', '기억남', '기억남는', '기억남는건', '기억상실증']


In [18]:
print("훈련 데이터 점수 : ", result.score(x_train, y_train))
print("테스트 데이터 점수 : ", result.score(x_test, y_test))

훈련 데이터 점수 :  0.8812266666666667
테스트 데이터 점수 :  0.85056


# LSTM을 이용한 분석

In [21]:
text_data, y_data = df_train['document'].values, df_train['label'].values

In [23]:
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
import numpy as np
import pickle
import os.path
#import keras.backend.tensorflow_backend as K
import tensorflow.keras.backend as K

# 토큰 파서
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

In [24]:
twitter_tag = Twitter()

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [25]:
cv = TfidfVectorizer(tokenizer=twitter_tokenizer)

In [26]:
!ls X*

ls: cannot access 'X*': No such file or directory


In [ ]:
# Tfidf 생성과 저장 - 단 파일이 없을 때만 !!
if not os.path.isfile("X_data.pickle"): 
    print('file does not exists')
    X_data = cv.fit_transform(text_data)
    pickle.dump(X_train, open("X_data.pickle", "wb"))

file does not exists


In [ ]:
# 저장된 tfidf vector 데이터 읽기
with open('X_data.pickle', 'rb') as f:
    X_data = pickle.load(f)

In [ ]:
!ls -al X*

-rw-rw-r-- 1 hjkim hjkim 24433014 Oct 28 18:39 X_data.pickle
-rw-rw-r-- 1 hjkim hjkim  7782344 Oct 28 19:03 X_test_1.pickle
-rw-rw-r-- 1 hjkim hjkim  5610600 Oct 28 18:56 X_test_2.pickle
-rw-rw-r-- 1 hjkim hjkim  8170628 Oct 28 18:40 X_test.pickle
-rw-rw-r-- 1 hjkim hjkim 23745856 Oct 28 19:02 X_train_1.pickle
-rw-rw-r-- 1 hjkim hjkim 14084384 Oct 28 18:55 X_train_2.pickle


In [ ]:
# one-hot encoding
Y_data = np_utils.to_categorical(y_data, 2)

In [ ]:
X_train = X_data[:100000]
X_test = X_data[100000:]

Y_train = Y_data[:100000]
Y_test = Y_data[100000:]

In [ ]:
max_words = 61070 
nb_classes = 2
batch_size = 1024
nb_epoch = 5

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
Y_train.shape[1]

(100000, 104160)
(50000, 104160)
(100000, 2)


2

In [ ]:
# LSTM 학습을 위한 데이터 재배열 (Time step)
X_train_rnn = X_train.A.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.A.reshape((X_test.shape[0], 1, X_test.shape[1]))

print(X_train_rnn.shape)
print(X_test_rnn.shape)

(100000, 1, 104160)
(50000, 1, 104160)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

def build_LSTM_model():
    model = Sequential()
    model.add(LSTM(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(Y_train.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
import keras.backend.tensorflow_backend as K
with K.tf.device('/GPU:1'):
    model_lstm = KerasClassifier(
    build_fn=build_LSTM_model, 
    epochs=nb_epoch, 
    batch_size=batch_size)
    
    model_lstm.fit(X_train_rnn, Y_train)

Epoch 1/5
100000/100000 [==============================] - 252s 3ms/step - loss: 0.5463 - acc: 0.7826
Epoch 2/5
100000/100000 [==============================] - 238s 2ms/step - loss: 0.2854 - acc: 0.8830
Epoch 3/5
100000/100000 [==============================] - 239s 2ms/step - loss: 0.2076 - acc: 0.9218
Epoch 4/5
100000/100000 [==============================] - 238s 2ms/step - loss: 0.1604 - acc: 0.9420
Epoch 5/5
100000/100000 [==============================] - 239s 2ms/step - loss: 0.1293 - acc: 0.9543


In [ ]:
y = model_lstm.predict(X_train_rnn)
y_train = y_data[:100000]
ac_score = metrics.accuracy_score(y_train, y)
print("훈련 셋 정답률 =", ac_score)

훈련 셋 정답률 = 0.96941


In [ ]:
# predict 함수는 예측 결과를 0 or 1로 출력하므로
# 학습과정에서 사용한 Y_train, Y_test 변수로 정확도 측정이 안됨
# Y_train, Y_test는 [0, 1], [1, 0]의 형태로 해당하는 감정 컬럼(class)은 1, 다른 컬럼은 0으로 표시됨
# 초기 y_data에 저장된 값을 그대로 활용하여 정확도를 측정

print("y : ", y)
print("Y_train[0] : ", Y_train)
print("y_train[0] : ", y_data)

y :  [0 1 0 ... 0 1 1]
Y_train[0] :  [[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]
y_train[0] :  [0 1 0 ... 0 1 0]


In [ ]:
y = model_lstm.predict(X_test_rnn)
y_test = y_data[100000:]
ac_score = metrics.accuracy_score(y_test, y)
print("테스트 셋 정답률 =", ac_score)

테스트 셋 정답률 = 0.83784
